<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Начнём с установок и импортов библиотек и функций.

Сначала установим библиотеку skopt, чтобы использовать их реализацию Байесовского подбора гиперпараметров.

In [1]:
!pip install scikit-optimize

Дальше импортируем всё остальное.

In [17]:
#импорт общих библиотек
import pandas as pd
import numpy as np
import time

#импорт библиотек для работы с естественным языком
import re
import spacy
language_model = spacy.load('en_core_web_sm', disable=["parser", "ner"])
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_set = stopwords.words('english')

#импортируем целевую метрику
from sklearn.metrics import f1_score

#импортируем модели
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier

#импортируем воспомогательные функции
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV
from skopt.callbacks import DeltaYStopper

#зададим случайное состояние
STATE = np.random.RandomState(1234)

pd.set_option('display.expand_frame_repr', False)
pd.options.display.max_colwidth = 100


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Посмотрим на наши данные.

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0


Уберём столбец Unnamed: 0 - явно дублирует индекс. Чьи-то шаловливые ручонки перезаписали датасет?

In [4]:
data = data.drop(columns = 'Unnamed: 0')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


Создадим функцию для очистки и лемматизации текста.

In [5]:
def lemm_and_clear(text):
    doc = language_model(text.lower())
    lemm = " ".join([token.lemma_ for token in doc])
    lemm = " ".join(re.sub(r'[^a-zA-Z ]', ' ', lemm).split())
    return lemm

Применим её к нашему корпусу.

In [6]:
data['lemm'] = data['text'].apply(lemm_and_clear)
data.head()

,text,toxic,lemm
0,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,explanation why the edit make under my username hardcore metallica fan be revert they be not van...
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,d aww he match this background colour I be seemingly stuck with thank talk january utc
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,hey man I be really not try to edit war it be just that this guy be constantly remove relevant i...
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,more I can not make any real suggestion on improvement I wonder if the section statistic should ...
4,"You, sir, are my hero. Any chance you remember what page that's on?",0,you sir be my hero any chance you remember what page that be on


Выведем на экран текст и его лемматизированный вариант для проверки качества лемматизации.

In [7]:
data[['text','lemm']].sample(n=1)

,text,lemm
71267,"""Welcome\n\nHello, and welcome to Wikipedia! Thank you for your contributions. I hope you like t...",welcome hello and welcome to wikipedia thank you for your contribution I hope you like the place...


Посмотрим, как распределены комментарии по токсичности.

In [8]:
data['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Выборка несбалансирована. Попробуем учесть при разделении на тренировочную и тестовую выборки.

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    data.drop(columns = 'toxic', axis=1),
    data['toxic'],
    test_size=0.2,
    random_state=STATE,
    stratify=data['toxic'] 
)


corpus_train = features_train['lemm']
corpus_test = features_test['lemm']
corpus_train

49231     question I believe that I ve understand and correctly use explain most of the historical indian ...
92758                                                               before return home and be disband in june
138496    I have respectfully append a reassurance to my comment that it be not a legal threat no legal ac...
2562      as I mention in the edit summary again do you even read it it be a personal communication hence ...
6830      hi dutchbloke I say might give reason to presume and hence have not allege that you be rex howev...
                                                         ...                                                 
63098     one will undoubtedly come out as soon as the demurrage fractionalize and I get the money transfe...
133901    I be reasonable with he they get a week s notice for an article that be be tag as unsourced for ...
63078                                                                where the four grouping be actually find
93821     

Теперь рассчитаем TF-IDF для каждого из слов корпусов и векторизуем их на основе этой величины.

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords_set)
tf_idf_train = count_tf_idf.fit_transform(corpus_train) 
tf_idf_test = count_tf_idf.transform(corpus_test) 

for stage in ['train','test']:
    print(f"Размер матрицы TF-IDF для выборки {stage}: {eval('_'.join(['tf_idf',stage])).shape}")

Размер матрицы TF-IDF для выборки train: (127433, 134531)
Размер матрицы TF-IDF для выборки test: (31859, 134531)


### Вывод:
- Данные проанализированы, признаки подготовлены к анализу.
- Ввиду количества фич фокус будет сделан на обучение и подбор параметров более простых моделей, ансамблевыми моделями пренебрежём по причине вычислительной сложности.

## Обучение

Напишем функцию для перебора гиперпараметров и сохранения результатов в словарь.

In [19]:
callback = DeltaYStopper(delta=0.05) 

In [20]:
def search_time_save(model, parameters, features = tf_idf_train,target = target_train):
    
    grid = BayesSearchCV(model, parameters, 
                         scoring = 'f1', cv = 2, verbose = 3, n_iter = 10, random_state = STATE)
    start = time.time()
    grid.fit(features, target, callback = callback)
    stop = time.time()
    
    learn_time = stop-start
    
    score = grid.best_score_
    print()
    print (f"Для выбранной модели наилучший результат = {score:0.4f} за {learn_time:0.2f} сек.")
    model_dict = {'score':score,
                  'learn_time':learn_time,
                  'fit_time':grid.refit_time_,
                  'params':grid.best_params_,
                  'model':grid.best_estimator_}
    
    return model_dict

Начнём с модели логистической регрессии, для которой будем перебирать параметр C. 

In [21]:
model = LogisticRegression()

parameters = {
    'C': [*range(1,100,5)],
    'solver':['saga'],
    'max_iter':[1000],
    'random_state': [STATE]
}

log_dict = search_time_save(model, parameters)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END C=41, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.757 total time=   7.3s
[CV 2/2] END C=41, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.761 total time=   6.9s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END C=41, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.757 total time=   6.6s
[CV 2/2] END C=41, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.761 total time=   6.5s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END C=31, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.756 total time=   5.9s
[CV 2/2] END C=31, max_iter=1000, random_state=RandomState(MT19937), solver=saga;, score=0.763 total time=   6.0s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END C=86, max_iter=1000, random_state=RandomState(MT19937), solver=saga

Теперь попробуем модель Ridge. 

In [22]:
model = RidgeClassifier()

parameters = {
    'alpha': np.arange(0.01, 1.0, 0.025),
    'solver':['sag'],
    'fit_intercept':[False],
    'random_state': [STATE]
}

ridge_dict = search_time_save(model,parameters)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END alpha=0.385, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.678 total time=   4.1s
[CV 2/2] END alpha=0.385, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.676 total time=   4.0s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END alpha=0.46, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.684 total time=   3.2s
[CV 2/2] END alpha=0.46, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.683 total time=   3.3s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END alpha=0.23500000000000001, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.664 total time=   6.2s
[CV 2/2] END alpha=0.23500000000000001, fit_intercept=False, random_state=RandomState(MT19937), solver=sag;, score=0.662 total time=   6.6s
Fitting 2 folds for each of 1 candidates,

Теперь обучим случайное дерево. По причине большого количества признаков будем искать нужное дерево среди деревьев большой глубины.

In [23]:
model =  DecisionTreeClassifier()

parameters = {'max_depth': [*range (50,151)],
             'random_state': [STATE]}

tree_dict = search_time_save(model,parameters)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=110, random_state=RandomState(MT19937);, score=0.727 total time=  31.2s
[CV 2/2] END max_depth=110, random_state=RandomState(MT19937);, score=0.725 total time=  29.6s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=128, random_state=RandomState(MT19937);, score=0.730 total time=  32.5s
[CV 2/2] END max_depth=128, random_state=RandomState(MT19937);, score=0.727 total time=  33.7s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=74, random_state=RandomState(MT19937);, score=0.725 total time=  22.6s
[CV 2/2] END max_depth=74, random_state=RandomState(MT19937);, score=0.718 total time=  22.3s
Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END max_depth=87, random_state=RandomState(MT19937);, score=0.726 total time=  25.3s
[CV 2/2] END max_depth=87, random_state=RandomState(MT19937);, score=0.722 total time=  24.7s
Fitting 

Сведём все результаты в единую табличку для наглядности.

In [24]:
dict_list = [log_dict, ridge_dict, tree_dict]

model_scores = pd.DataFrame(dict_list, index = ['log', 'ridge','tree']).drop(columns=['params','model'])\
.sort_values(by='score', ascending = False)
model_scores

,score,learn_time,fit_time
log,0.759532,88.184498,14.046810
tree,0.728175,322.403913,64.245566
ridge,0.696389,68.322099,5.799469


Выберем лучшую из моделей. По совокупности скорости и точности - это модель линейной регрессии. К тому же, она единственная показала требуемый f1-score.

In [25]:
best_model = eval('_'.join([model_scores[model_scores['score'] == model_scores['score'].max()].index.tolist()[0],
                           'dict']))['model']

Протестируем модель на тестовой выборке, чтобы проверить её на переобучение.

In [26]:
predict = best_model.predict(tf_idf_test)
score = f1_score(target_test,predict)
print(f"F1-score на тестовой выборке = {score:0.4f}")

F1-score на тестовой выборке = 0.7747


Результат на тестовой выборке также выше порогового значение (>0.75). Модель можно использовать.  

Для наглядности выведем параметры модели.

In [27]:
eval('_'.join([model_scores[model_scores['score'] == model_scores['score'].max()].index.tolist()[0],
                           'dict']))['params']

OrderedDict([('C', 31),
             ('max_iter', 1000),
             ('random_state', RandomState(MT19937) at 0x7F1561B27E40),
             ('solver', 'saga')])

## Выводы

- Были проанализированы исходные данные, отброшены лишние столбцы.
- Исходные комментарии были очищены от знаков препинания и лемматизированы.
- С помощью TF-IDF были подготовлены признаки для машинного обучения.
- С помощью BayesSearchCV были подобраны гиперпараметры для трёх моделей.
- Наилучшие результаты показала модель линейной регрессии c С = 26.